### In this example, we will explain how the binary columns are integrated in MOMRollRateTable

In [1]:
import polars as pl

data_i = pl.scan_csv("../tests/simulation_data/test_sample_0.csv").collect()
data_i_1 = pl.scan_csv("../tests/simulation_data/test_sample_1.csv").collect()

In [2]:
data_i.head()

id,delq,Open,Active,Deactive,Closed,bin_ind_1,bin_ind_2
str,i64,i64,i64,i64,i64,i64,i64
"""950165FF0FFE88…",0,1,1,0,0,0,1
"""885857A5D45526…",0,1,1,0,0,0,1
"""CC2672DFA2988C…",0,1,1,0,0,1,0
"""D73B2D86E08D8E…",0,1,1,0,0,1,0
"""23D46CDE09B973…",1,1,1,0,0,1,0


In [3]:
data_i_1.head()

id,delq,Open,Active,Deactive,Closed,bin_ind_1,bin_ind_2
str,i64,i64,i64,i64,i64,i64,i64
"""B1BF9E2890A8D8…",1,1,1,0,0,1,0
"""6EDAD278689D5E…",1,1,1,0,0,1,1
"""8779CF51DFA115…",1,1,1,0,0,1,0
"""81EA835BB1B98E…",0,1,1,0,0,0,0
"""691C4F9084FC3A…",0,1,1,0,0,1,1


We can see that there are 8 columns:

**id**: Account id, the unique key of the dataset. \
**delq**: The delinquency of each account. \
**bin_ind_1**: A binary indicator. \
**bin_ind_2**: A binary indicator. \
**Open, Active, Deactive, Closed**: Indicates if the account is open, active, deactive or closed in that month. 

The last 4 columns are not of any use to us for this example.

Suppose now that we want to include the 2 binary columns in our data into our roll rate table. Before just adding them into the MOMRollRateTable object we have to consider to which indicator we want to give priority, i.e. if an account id is populated with both indicators who do we consider more important? 

In [4]:
from roll_rate_analysis import MOMRollRateTable

table = MOMRollRateTable(
    unique_key_col="id",
    delinquency_col="delq",
    path_i="../tests/simulation_data/test_sample_0.csv",
    path_i_1="../tests/simulation_data/test_sample_1.csv",
    max_delq=6,
    binary_cols=["bin_ind_1", "bin_ind_2"],
)

The rows that we put first have the largest priority, in other words, priority is in descending order [bin_ind_1 > bin_ind_2]. That said, for accounts on row "bin_ind_2", the bin_ind_1 variable **equal to zero**. But accounts on row "bin_ind_1" could have **both indicators equal to 1 or just the bin_ind_1**.

In [5]:
table.build()

In [6]:
table.get_roll_rates()

,0_cycle_deliqnuent,1_cycle_deliqnuent,2_cycle_deliqnuent,3_cycle_deliqnuent,4_cycle_deliqnuent,5_cycle_deliqnuent,6+_cycle_deliqnuent,bin_ind_2,bin_ind_1
0_cycle_deliqnuent,1816,401,1,0,0,0,0,2216,4374
1_cycle_deliqnuent,429,618,139,0,0,0,0,1158,2477
2_cycle_deliqnuent,58,159,21,51,0,0,0,307,585
3_cycle_deliqnuent,18,18,6,2,30,0,0,76,156
4_cycle_deliqnuent,6,3,2,0,1,17,0,27,49
5_cycle_deliqnuent,2,3,0,0,0,0,9,12,30
6+_cycle_deliqnuent,3,0,0,0,0,0,24,32,50
bin_ind_2,2352,1113,169,40,38,23,31,3994,7669
bin_ind_1,4894,2313,367,84,61,27,74,7624,15287


Also, by default the indicators are prioritized from the delinquency columns. \
For example if an account is 2 cycle delinquent and both indicator columns are equal to 1, then the will appear on the row of the biggest priority indicator, meaning bin_ind_1.